In [ ]:
import os
import folium

import json
import pandas as pd

print(folium.__version__)

# Using `folium.colormap`

**A few examples of how to use `folium.colormap` in choropleths.**

Let's load a GeoJSON file, and try to choropleth it.

In [ ]:

us_states = os.path.join('data', 'us-states.json')
US_Unemployment_Oct2012 = os.path.join(
    'data', 'US_Unemployment_Oct2012.csv'
)

geo_json_data = json.load(open(us_states))
unemployment = pd.read_csv(US_Unemployment_Oct2012)

unemployment_dict = unemployment.set_index('State')['Unemployment']

## Self-defined

You can build a choropleth in using a self-defined function.
It has to output an hexadecimal color string of the form `#RRGGBB` or `#RRGGBBAA`.

In [ ]:
def my_color_function(feature):
    """Maps low values to green and hugh values to red."""
    if unemployment_dict[feature['id']] > 6.5:
        return '#ff0000'
    else:
        return '#008000'

In [ ]:
m = folium.Map([43, -100], tiles='cartodbpositron', zoom_start=4)

folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': my_color_function(feature),
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)

#m.save(os.path.join('results', 'Colormaps_0.html'))

m

## StepColormap

But to help you define you colormap, we've embedded `StepColormap` in `folium.colormap`.

You can simply define the colors you want, and the `index` (*thresholds*) that correspond.

In [ ]:
import branca.colormap as cm


step = cm.StepColormap(
    ['green', 'yellow', 'red'],
    vmin=3, vmax=10,
    index=[3, 4, 8, 10],
    caption='step'
)

step

In [ ]:
m = folium.Map([43, -100], tiles='cartodbpositron', zoom_start=4)

folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': step(unemployment_dict[feature['id']]),
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)

#m.save(os.path.join('results', 'Colormaps_1.html'))

m

If you specify no index, colors will be set uniformely.

In [ ]:
cm.StepColormap(['r', 'y', 'g', 'c', 'b', 'm'])

## LinearColormap

But sometimes, you would prefer to have a *continuous* set of colors. This can be done by `LinearColormap`.

In [ ]:
linear = cm.LinearColormap(
    ['green', 'yellow', 'red'],
    vmin=3, vmax=10
)

linear

In [ ]:
m = folium.Map([43, -100], tiles='cartodbpositron', zoom_start=4)

folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': linear(unemployment_dict[feature['id']]),
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)

#m.save(os.path.join('results', 'Colormaps_2.html'))

m

Again, you can set the `index` if you want something irregular.

In [ ]:
cm.LinearColormap(
    ['red', 'orange', 'yellow', 'green'],
    index=[0, 0.1, 0.9, 1.0]
)

If you want to transform a linear map into a *step* one, you can use the method `to_step`.

In [ ]:
linear.to_step(6)

You can also use more sophisticated rules to create the thresholds.

In [ ]:
linear.to_step(
    n=6,
    data=[30.6, 50, 51, 52, 53, 54, 55, 60, 70, 100],
    method='quantiles',
    round_method='int'
)

And the opposite is also possible with `to_linear`.

In [ ]:
step.to_linear()

## Build-in

For convenience, we provide a (small) set of built-in linear colormaps, in `folium.colormap.linear`.

In [ ]:
cm.linear.OrRd_09

You can also use them to generate regular `StepColormap`.

In [ ]:
cm.linear.PuBuGn_09.to_step(12)

Of course, you may need to scale the colormaps to your bounds. This is doable with `.scale`.

In [ ]:
cm.linear.YlGnBu_09.scale(3, 12)

In [ ]:
cm.linear.RdGy_11.to_step(10).scale(5, 100)

At last, if you want to check them all, simply ask for `linear` in the notebook.

In [ ]:
cm.linear

## Draw a `ColorMap` on a map

By the way, a ColorMap is also a Folium `Element` that you can draw on a map.

In [ ]:
m = folium.Map(tiles='cartodbpositron')

colormap = cm.linear.Set1_09.scale(0, 35).to_step(10)
colormap.caption = 'A colormap caption'
m.add_child(colormap)

#m.save(os.path.join('results', 'Colormaps_3.html'))

m